In [ ]:
work_directory = "/Volumes/T7 Red/wsi_code"

In [ ]:
import os
import sys
import matplotlib.pyplot as plt

# Add the path to your project root directory
if work_directory not in sys.path:
    sys.path.append(work_directory)
# my utility functions

# ----------------------------
import h5py

In [ ]:
# Define data directories
DATASETS_PATH = os.path.join(work_directory, "datasets")
PROCESSED_PATH = os.path.join(DATASETS_PATH, "processed")
INTERIM_PATH = os.path.join(DATASETS_PATH, "interim")

ls_patchs_path = os.path.join(INTERIM_PATH, "ls_patchs_384")
lms_patchs_path = os.path.join(INTERIM_PATH, "lms_patchs_384")
mpskt_patchs_path = os.path.join(INTERIM_PATH, "mpskt_patchs_384")
ss_patchs_path = os.path.join(INTERIM_PATH, "ss_patchs_384")

hdf5_path = os.path.join(PROCESSED_PATH, "patchs_384_40k.hdf5")

In [ ]:
# Open the HDF5 file for reading
file = h5py.File(hdf5_path, "r")
file["00/01/images"].shape

In [ ]:
keys_list = []
file.visit(lambda name: keys_list.append(name) if "images" in name else None)
print(len(keys_list), keys_list)

In [ ]:
# Define the number of rows and columns for the subplot grid
num_rows = 4
num_columns = 5

# Calculate the total number of subplots
total_subplots = num_rows * num_columns

# Create a single figure for all images with the specified subplot layout
fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 12))

for i, key in enumerate(keys_list[:total_subplots]):
    img = file[key][0]
    mask = file[key][1]

    row_idx = i // num_columns
    col_idx = i % num_columns

    axes[row_idx, col_idx].imshow(img, cmap="gray")
    axes[row_idx, col_idx].axis("off")
    axes[row_idx, col_idx].imshow(mask, cmap="gray")
    axes[row_idx, col_idx].axis("off")

    # Set title using the key
    axes[row_idx, col_idx].set_title(key)

# close the HDF5 file
file.close()

# Ensure proper spacing between subplots
plt.tight_layout()

# Show the combined figure with all images
plt.show()

In [ ]:
def select_case_data(file_path, selected_case):
    # Open the HDF5 file
    file = h5py.File(file_path, "r")

    # Initialize lists to store data
    selected_images = []
    selected_labels = []
    remaining_images = []
    remaining_labels = []

    # Iterate through the keys
    for i in range(4):  # Assuming you have four illnesses (00, 01, 02, 03)
        for j in range(
            5
        ):  # Assuming you have five cases in each illness (00, 01, 02, 03, 04)
            key = f"{i:02d}/{j:02d}"

            if j == selected_case:
                selected_images.extend(file[key + "/images"][:])
                selected_labels.extend(file[key + "/labels"][:])
            else:
                remaining_images.extend(file[key + "/images"][:])
                remaining_labels.extend(file[key + "/labels"][:])

    # Close the HDF5 file when done
    file.close()

    return selected_images, selected_labels, remaining_images, remaining_labels

In [ ]:
selected_case = 1
selected_images, selected_labels, remaining_images, remaining_labels = select_case_data(
    hdf5_path, selected_case
)
print(
    len(selected_images),
    len(selected_labels),
    len(remaining_images),
    len(remaining_labels),
)

In [ ]:
selected_cases = [0, 2000, 4000, 6000]

# Show images from the selected cases in the same plot
fig, axes = plt.subplots(
    1, len(selected_cases), figsize=(15, 5)
)  # Create subplots for each case

for i, ax in enumerate(axes):
    ax.imshow(selected_images[selected_cases[i]], cmap="gray")
    ax.set_title(
        f"Case {selected_cases[i]} with label {selected_labels[selected_cases[i]]}"
    )
    ax.axis("off")

plt.show()